In [ ]:
#設定讀取資料的年份與時段

year = 104    #110年或104年
rush_hour = "晨峰"    #晨峰或昏峰

inputLocation = f"{year}{rush_hour}_禁左/"

In [ ]:
import pandas as pd

#open 校估.xlsx
#for sheet 1: link, AM, PM, A, B
#it stands for link name, AM flow, PM flow, A node, B node
#for sheet 2: link, AM, PM, A1, B1, A2, B2
#it stands for link name, AM flow, PM flow, A1 node, B1 node, A2 node, B2 node

#open UE_results.dat
#use (A, B) as key to find the corresponding link name in 校估.xlsx and compare AM flow with UE_flow

# df= pd.read_excel("校估.xlsx", sheet_name=None)
df= pd.read_excel("校估.xlsx", sheet_name=None)


sheet1 = df["工作表1"]
sheet2 = df["工作表2"]
# sheet3 = df["工作表3"]

ue_results = pd.read_csv(inputLocation + f"{year}{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)
sheet1["link_name"] = sheet1.apply(lambda row: f"{row['A']}_{row['B']}", axis=1)

#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], on="link_name", how="left")
#reserve the link,AM,UE_flow
sheet1 = sheet1[["group", "link", "AM", "UE_flow", "capacity"]]
#calculate the difference percentage
sheet1["diff"] = (-sheet1["AM"] + sheet1["UE_flow"]) / sheet1["AM"] * 100
sheet1["diff"] = sheet1["diff"].round(2)
sheet1["diff"] = sheet1["diff"].astype(str) + "%"

#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet2["link_name1"] = sheet2.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet2["link_name2"] = sheet2.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet2 = sheet2.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet2 = sheet2.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row
sheet2["UE_flow"] = sheet2["UE_flow_1"] + sheet2["UE_flow_2"]
sheet2["capacity"] = sheet2["capacity_1"] + sheet2["capacity_2"]
#calculate the difference percentage
sheet2["diff"] = (-sheet2["AM"] + sheet2["UE_flow"]) / sheet2["AM"] * 100
sheet2["diff"] = sheet2["diff"].round(2)
sheet2["diff"] = sheet2["diff"].astype(str) + "%"
#reserve the link,AM,UE_flow
sheet2 = sheet2[["group","link", "AM", "UE_flow", "capacity", "diff"]]

# #for sheet3, we need to find the corresponding link name for A1, B1; A2, B2; A3, B3
# sheet3["link_name1"] = sheet3.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
# sheet3["link_name2"] = sheet3.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
# sheet3["link_name3"] = sheet3.apply(lambda row: f"{int(row['A3'])}_{int(row['B3'])}", axis=1)
# sheet3 = sheet3.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
# sheet3 = sheet3.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
# sheet3 = sheet3.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name3", right_on="link_name", how="left")
# #rename the columns
# sheet3.rename(columns={"UE_flow": "UE_flow_3", "capacity": "capacity_3"}, inplace=True)
# #calculate the total flow and capacity for this row
# sheet3["UE_flow"] = sheet3["UE_flow_1"] + sheet3["UE_flow_2"] + sheet3["UE_flow_3"]
# sheet3["capacity"] = sheet3["capacity_1"] + sheet3["capacity_2"] + sheet3["capacity_3"]
# #calculate the difference percentage
# sheet3["diff"] = (-sheet3["AM"] + sheet3["UE_flow"]) / sheet3["AM"] * 100
# sheet3["diff"] = sheet3["diff"].round(2)
# sheet3["diff"] = sheet3["diff"].astype(str) + "%"
# #reserve the link,AM,UE_flow
# sheet3 = sheet3[["link", "AM", "UE_flow", "capacity", "diff"]]

#now,we can combine the two sheets
sheet1 = pd.concat([sheet1, sheet2], ignore_index=True)
# sheet1 = pd.concat([sheet1, sheet3], ignore_index=True)

# filter out groups
# filtered_groups = ["中山往東", "中山往西", "參考", "民族往南", "民族往北","外雙溪出城","外雙溪進城","淡水出","淡水進"]#,"基隆河進城","基隆河出城"
# filtered_groups = [ "參考", "外雙溪出城","外雙溪進城","淡水出","淡水進","基隆河進城","基隆河出城","淡水河進城","淡水河出城"]#"中山往東", "中山往西","民族往南", "民族往北",
# filtered_groups = [ "參考","淡水出","淡水進","基隆河進城","基隆河出城","淡水河進城","淡水河出城","中山往東", "中山往西","民族往南", "民族往北"]#,"外雙溪出城","外雙溪進城"
filtered_groups = ["參考", "外雙溪出城","外雙溪進城"]
sheet1 = sheet1[~sheet1["group"].isin(filtered_groups)]

#calculate the mape for the whole data from diff
sheet1["diff_abs"] = sheet1["diff"].str.replace("%", "").astype(float).abs()
#V/C ratio = UE_flow / capacity
sheet1["V/C"] = sheet1["UE_flow"] / sheet1["capacity"]
#GEH = (2(UE_flow - AM)^2 / (UE_flow + AM))^0.5
sheet1["GEH"] = (2 * (sheet1["UE_flow"] - sheet1["AM"]) ** 2 / (sheet1["UE_flow"] + sheet1["AM"])) ** 0.5
#average the diff_abs and print the result
mape = sheet1["diff_abs"].mean()
#round the UE_flow, V/C and capacity to int
sheet1["capacity"] = sheet1["capacity"].astype(int)
# sheet1["capacity"] = sheet1["capacity"].round(0).astype(int)
sheet1["UE_flow"] = sheet1["UE_flow"].round(0).astype(int)
sheet1["V/C"] = sheet1["V/C"].round(2)
print("MAPE: ", mape.round(2), "%")

# in sheet1, for each 'group' (see the 'group' column), we can calculate the sum of UE_flow, and the sum of AM flow

group_names = sheet1['group'].unique()
groups = {i: {'AM_flow': 0, 'UE_flow': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['AM_flow'] += row['AM']
    groups[group]['UE_flow'] += row['UE_flow']
# Now we can print the total AM flow and UE flow for each group
for group, flows in groups.items():
    print(f"""【{group}】 
    AM flow = {flows['AM_flow']}, 
    UE flow = {flows['UE_flow']}, 
    diff = {(flows['UE_flow'] - flows['AM_flow'])/ flows['AM_flow'] * 100:.2f}%""")

#we can conduct a simple statistical analysis to see if the distribution of the flow is different
import scipy.stats as stats
#compare the distribution of the flow between AM and UE_flow for each group
def ks_test(am_flow, ue_flow):
    #use ks test to compare the distribution
    ks_statistic, p_value = stats.ks_2samp(am_flow, ue_flow)
    print("\tKS test statistic: ", ks_statistic)
    print("\tp-value: ", p_value)
    if p_value < 0.05:
        print("\t拒絕虛無假設：AM flow 與 UE flow 的分佈不同")
    else:
        print("\t接受虛無假設：AM flow 與 UE flow 的分佈相同")

#apply ks_test to each group
for group in group_names:
    group_data = sheet1[sheet1['group'] == group]
    am_flow = group_data['AM'].values
    ue_flow = group_data['UE_flow'].values
    print(f"【{group}】")
    ks_test(am_flow, ue_flow)
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "capacity", "AM", "UE_flow", "V/C", "diff", "GEH"]]
#show all the results in a table


In [ ]:
import pandas as pd

#open 校估.xlsx
#for sheet 1: link, AM, PM, A, B
#it stands for link name, AM flow, PM flow, A node, B node
#for sheet 2: link, AM, PM, A1, B1, A2, B2
#it stands for link name, AM flow, PM flow, A1 node, B1 node, A2 node, B2 node

#open UE_results.dat
#use (A, B) as key to find the corresponding link name in 校估.xlsx and compare AM flow with UE_flow

# df= pd.read_excel("校估.xlsx", sheet_name=None)
df= pd.read_excel("校估112-113.xlsx", sheet_name=None)

sheet1 = df["工作表1"]
sheet2 = df["工作表2"]
sheet3 = df["工作表3"]

ue_results = pd.read_csv(inputLocation + f"{year}{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)
sheet1["link_name"] = sheet1.apply(lambda row: f"{row['A']}_{row['B']}", axis=1)

#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], on="link_name", how="left")
#reserve the link,AM,UE_flow
sheet1 = sheet1[["group", "link", "AM", "UE_flow", "capacity"]]
#calculate the difference percentage
sheet1["diff"] = (-sheet1["AM"] + sheet1["UE_flow"]) / sheet1["AM"] * 100
sheet1["diff"] = sheet1["diff"].round(2)
sheet1["diff"] = sheet1["diff"].astype(str) + "%"

#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet2["link_name1"] = sheet2.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet2["link_name2"] = sheet2.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet2 = sheet2.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet2 = sheet2.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row
sheet2["UE_flow"] = sheet2["UE_flow_1"] + sheet2["UE_flow_2"]
sheet2["capacity"] = sheet2["capacity_1"] + sheet2["capacity_2"]
#calculate the difference percentage
sheet2["diff"] = (-sheet2["AM"] + sheet2["UE_flow"]) / sheet2["AM"] * 100
sheet2["diff"] = sheet2["diff"].round(2)
sheet2["diff"] = sheet2["diff"].astype(str) + "%"
#reserve the link,AM,UE_flow
sheet2 = sheet2[["group","link", "AM", "UE_flow", "capacity", "diff"]]

# #for sheet3, we need to find the corresponding link name for A1, B1; A2, B2; A3, B3
sheet3["link_name1"] = sheet3.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet3["link_name2"] = sheet3.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet3["link_name3"] = sheet3.apply(lambda row: f"{int(row['A3'])}_{int(row['B3'])}", axis=1)
sheet3 = sheet3.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet3 = sheet3.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
sheet3 = sheet3.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name3", right_on="link_name", how="left")
#rename the columns
sheet3.rename(columns={"UE_flow": "UE_flow_3", "capacity": "capacity_3"}, inplace=True)
#calculate the total flow and capacity for this row
sheet3["UE_flow"] = sheet3["UE_flow_1"] + sheet3["UE_flow_2"] + sheet3["UE_flow_3"]
sheet3["capacity"] = sheet3["capacity_1"] + sheet3["capacity_2"] + sheet3["capacity_3"]
#calculate the difference percentage
sheet3["diff"] = (-sheet3["AM"] + sheet3["UE_flow"]) / sheet3["AM"] * 100
sheet3["diff"] = sheet3["diff"].round(2)
sheet3["diff"] = sheet3["diff"].astype(str) + "%"
#reserve the link,AM,UE_flow
sheet3 = sheet3[["group","link", "AM", "UE_flow", "capacity", "diff"]]

#now,we can combine the two sheets
sheet1 = pd.concat([sheet1, sheet2], ignore_index=True)
sheet1 = pd.concat([sheet1, sheet3], ignore_index=True)

# filter out groups
filtered_groups = ["中山往東", "中山往西", "參考", "民族往南", "民族往北"]
sheet1 = sheet1[~sheet1["group"].isin(filtered_groups)]

#calculate the mape for the whole data from diff
sheet1["diff_abs"] = sheet1["diff"].str.replace("%", "").astype(float).abs()
#V/C ratio = UE_flow / capacity
sheet1["V/C"] = sheet1["UE_flow"] / sheet1["capacity"]
#GEH = (2(UE_flow - AM)^2 / (UE_flow + AM))^0.5
sheet1["GEH"] = (2 * (sheet1["UE_flow"] - sheet1["AM"]) ** 2 / (sheet1["UE_flow"] + sheet1["AM"])) ** 0.5
#average the diff_abs and print the result
mape = sheet1["diff_abs"].mean()
#round the UE_flow, V/C and capacity to int
sheet1["capacity"] = sheet1["capacity"].astype(int)
# sheet1["capacity"] = sheet1["capacity"].round(0).astype(int)
sheet1["UE_flow"] = sheet1["UE_flow"].round(0).astype(int)
sheet1["V/C"] = sheet1["V/C"].round(2)
print("MAPE: ", mape.round(2), "%")

# in sheet1, for each 'group' (see the 'group' column), we can calculate the sum of UE_flow, and the sum of AM flow

group_names = sheet1['group'].unique()
groups = {i: {'AM_flow': 0, 'UE_flow': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['AM_flow'] += row['AM']
    groups[group]['UE_flow'] += row['UE_flow']
# Now we can print the total AM flow and UE flow for each group
for group, flows in groups.items():
    print(f"""【{group}】 
    AM flow = {flows['AM_flow']}, 
    UE flow = {flows['UE_flow']}, 
    diff = {(flows['UE_flow'] - flows['AM_flow'])/ flows['AM_flow'] * 100:.2f}%""")

#we can conduct a simple statistical analysis to see if the distribution of the flow is different
import scipy.stats as stats
#compare the distribution of the flow between AM and UE_flow for each group
def ks_test(am_flow, ue_flow):
    #use ks test to compare the distribution
    ks_statistic, p_value = stats.ks_2samp(am_flow, ue_flow)
    print("\tKS test statistic: ", ks_statistic)
    print("\tp-value: ", p_value)
    if p_value < 0.05:
        print("\t拒絕虛無假設：AM flow 與 UE flow 的分佈不同")
    else:
        print("\t接受虛無假設：AM flow 與 UE flow 的分佈相同")

#apply ks_test to each group
for group in group_names:
    group_data = sheet1[sheet1['group'] == group]
    am_flow = group_data['AM'].values
    ue_flow = group_data['UE_flow'].values
    print(f"【{group}】")
    ks_test(am_flow, ue_flow)
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
sheet1[["group", "link", "capacity", "AM", "UE_flow", "V/C", "diff", "GEH"]]